/usr/local/anaconda3/envs/snspark/bin/pip


## Setup environment

In [2]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate() 
spark

In [5]:
myRange = spark.range(1000).toDF("number")

In [6]:
divisBy2 = myRange.where("number % 2 = 0")

In [7]:
flightData2015 = spark\
  .read\
  .option("inferSchema", "true")\
  .option("header", "true")\
  .csv("../data/flight-data/csv/2015-summary.csv")

In [8]:
flightData2015.show()

+--------------------+-------------------+-----+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+--------------------+-------------------+-----+
|       United States|            Romania|   15|
|       United States|            Croatia|    1|
|       United States|            Ireland|  344|
|               Egypt|      United States|   15|
|       United States|              India|   62|
|       United States|          Singapore|    1|
|       United States|            Grenada|   62|
|          Costa Rica|      United States|  588|
|             Senegal|      United States|   40|
|             Moldova|      United States|    1|
|       United States|       Sint Maarten|  325|
|       United States|   Marshall Islands|   39|
|              Guyana|      United States|   64|
|               Malta|      United States|    1|
|            Anguilla|      United States|   41|
|             Bolivia|      United States|   30|
|       United States|           Paraguay|    6|
|             Algeri

In [9]:
flightData2015.createOrReplaceTempView("flight_data_2015")

In [10]:
sqlWay = spark.sql("""
SELECT DEST_COUNTRY_NAME, count(1)
FROM flight_data_2015
GROUP BY DEST_COUNTRY_NAME
""")

dataFrameWay = flightData2015\
  .groupBy("DEST_COUNTRY_NAME")\
  .count()

In [11]:
sqlWay.explain()
dataFrameWay.explain()

== Physical Plan ==
*(2) HashAggregate(keys=[DEST_COUNTRY_NAME#47], functions=[count(1)])
+- Exchange hashpartitioning(DEST_COUNTRY_NAME#47, 200)
   +- *(1) HashAggregate(keys=[DEST_COUNTRY_NAME#47], functions=[partial_count(1)])
      +- *(1) FileScan csv [DEST_COUNTRY_NAME#47] Batched: false, Format: CSV, Location: InMemoryFileIndex[file:/Users/tallamjr/github/forks/Spark-The-Definitive-Guide/data/flight-data/c..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string>
== Physical Plan ==
*(2) HashAggregate(keys=[DEST_COUNTRY_NAME#47], functions=[count(1)])
+- Exchange hashpartitioning(DEST_COUNTRY_NAME#47, 200)
   +- *(1) HashAggregate(keys=[DEST_COUNTRY_NAME#47], functions=[partial_count(1)])
      +- *(1) FileScan csv [DEST_COUNTRY_NAME#47] Batched: false, Format: CSV, Location: InMemoryFileIndex[file:/Users/tallamjr/github/forks/Spark-The-Definitive-Guide/data/flight-data/c..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY

In [12]:
from pyspark.sql.functions import max

flightData2015.select(max("count")).take(1)

[Row(max(count)=370002)]

In [13]:
maxSql = spark.sql("""
SELECT DEST_COUNTRY_NAME, sum(count) as destination_total
FROM flight_data_2015
GROUP BY DEST_COUNTRY_NAME
ORDER BY sum(count) DESC
LIMIT 5
""")

maxSql.show()

+-----------------+-----------------+
|DEST_COUNTRY_NAME|destination_total|
+-----------------+-----------------+
|    United States|           411352|
|           Canada|             8399|
|           Mexico|             7140|
|   United Kingdom|             2025|
|            Japan|             1548|
+-----------------+-----------------+



In [14]:
from pyspark.sql.functions import desc

flightData2015\
  .groupBy("DEST_COUNTRY_NAME")\
  .sum("count")\
  .withColumnRenamed("sum(count)", "destination_total")\
  .sort(desc("destination_total"))\
  .limit(5)\
  .show()

+-----------------+-----------------+
|DEST_COUNTRY_NAME|destination_total|
+-----------------+-----------------+
|    United States|           411352|
|           Canada|             8399|
|           Mexico|             7140|
|   United Kingdom|             2025|
|            Japan|             1548|
+-----------------+-----------------+



In [15]:
flightData2015\
  .groupBy("DEST_COUNTRY_NAME")\
  .sum("count")\
  .withColumnRenamed("sum(count)", "destination_total")\
  .sort(desc("destination_total"))\
  .limit(5)\
  .explain()

== Physical Plan ==
TakeOrderedAndProject(limit=5, orderBy=[destination_total#132L DESC NULLS LAST], output=[DEST_COUNTRY_NAME#47,destination_total#132L])
+- *(2) HashAggregate(keys=[DEST_COUNTRY_NAME#47], functions=[sum(cast(count#49 as bigint))])
   +- Exchange hashpartitioning(DEST_COUNTRY_NAME#47, 200)
      +- *(1) HashAggregate(keys=[DEST_COUNTRY_NAME#47], functions=[partial_sum(cast(count#49 as bigint))])
         +- *(1) FileScan csv [DEST_COUNTRY_NAME#47,count#49] Batched: false, Format: CSV, Location: InMemoryFileIndex[file:/Users/tallamjr/github/forks/Spark-The-Definitive-Guide/data/flight-data/c..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string,count:int>
